# Selenium para Ciencia de Datos

#### Borja Barber Lead Instructor DS

##  Índice
1. [Introducción](#introduccion)
2. [Instalación y Configuración](#instalacion)
3. [Conceptos Básicos](#conceptos-basicos)
4. [Navegación Web](#navegacion)
5. [Localización de Elementos](#localizacion)
6. [Extracción de Datos](#extraccion)
7. [Web Scraping Avanzado](#scraping-avanzado)
8. [Caso Práctico Completo](#caso-practico)
9. [Mejores Prácticas](#mejores-practicas)

---

## 1. Introducción <a name="introduccion"></a>

### ¿Qué es Selenium?

Selenium es una herramienta de automatización de navegadores web que permite:
- **Automatizar** interacciones con páginas web
- **Extraer datos** de sitios web dinámicos (JavaScript)
- **Simular** comportamiento humano (clicks, scroll, formularios)
- **Recopilar datos** para análisis y ciencia de datos

### ¿Por qué usar Selenium en Ciencia de Datos?

- Muchas páginas web modernas cargan datos con JavaScript (AJAX)
- BeautifulSoup y requests no pueden ejecutar JavaScript
- Selenium permite acceder a datos que se cargan dinámicamente
- Ideal para crear datasets personalizados mediante web scraping

---

## 2. Instalación y Configuración <a name="instalacion"></a>

### Paso 1: Instalar Selenium

Ejecuta el siguiente comando para instalar Selenium:

In [ ]:
# ========================================
# CELDA 1: INSTALACIÓN DE SELENIUM
# ========================================

# El símbolo ! ejecuta comandos del sistema operativo desde Jupyter
# pip es el gestor de paquetes de Python que instala librerías

# Instalar Selenium - La librería principal para automatizar navegadores
# Selenium permite controlar Chrome, Firefox, Safari, etc. mediante código Python
!pip install selenium

# Instalar Pandas - Para manipular y analizar datos en formato tabular (DataFrames)
# Pandas es esencial en ciencia de datos para trabajar con datos estructurados
!pip install pandas

# Instalar NumPy - Para operaciones numéricas y arrays
# NumPy es la base del ecosistema científico de Python
!pip install numpy

# NOTA PEDAGÓGICA:
# Estas instalaciones solo necesitan ejecutarse UNA VEZ en tu entorno.
# Si ya las tienes instaladas, verás un mensaje indicando que ya existen.

### Paso 2: Instalar WebDriver Manager

WebDriver Manager descarga automáticamente el driver del navegador:

**⚠️ NOTA IMPORTANTE SOBRE SELECTORES:**
Los sitios web cambian frecuentemente su estructura HTML. Si encuentras errores como `NoSuchElementException`, significa que el selector CSS o XPath ya no es válido. En el tutorial aprenderás cómo:
- Usar try-except para manejar estos casos
- Tener selectores alternativos como plan B
- Verificar selectores usando las DevTools del navegador (F12)

In [ ]:
# ========================================
# CELDA 2: INSTALACIÓN DE WEBDRIVER MANAGER
# ========================================

# WebDriver Manager es una herramienta que SIMPLIFICA enormemente trabajar con Selenium
# 
# ¿QUÉ PROBLEMA RESUELVE?
# Antes, tenías que:
# 1. Descargar manualmente el driver del navegador (chromedriver.exe)
# 2. Colocarlo en una ruta específica
# 3. Actualizar manualmente cada vez que el navegador se actualiza
#
# WebDriver Manager hace todo esto AUTOMÁTICAMENTE por ti
# Detecta tu navegador, descarga el driver correcto y lo gestiona

!pip install webdriver-manager

# VENTAJAS PARA CIENCIA DE DATOS:
# - Código portable: funciona en cualquier máquina sin configuración manual
# - Siempre actualizado: descarga la versión compatible con tu navegador
# - Menos errores: evita problemas de incompatibilidad de versiones

### Paso 3: Importar librerías necesarias

In [ ]:
# ========================================
# CELDA 3: IMPORTAR LIBRERÍAS NECESARIAS
# ========================================

# --- IMPORTACIONES DE SELENIUM ---

# webdriver: El módulo principal que controla el navegador
from selenium import webdriver

# By: Clase que define CÓMO buscar elementos en la página
# Ejemplo: By.ID, By.CSS_SELECTOR, By.XPATH, etc.
from selenium.webdriver.common.by import By

# Keys: Simula teclas del teclado (Enter, Tab, flechas, etc.)
# Útil para enviar formularios o navegar con teclado
from selenium.webdriver.common.keys import Keys

# WebDriverWait: Implementa ESPERAS INTELIGENTES
# Espera hasta que se cumpla una condición (elemento visible, clickeable, etc.)
from selenium.webdriver.support.ui import WebDriverWait

# expected_conditions (EC): Condiciones predefinidas para usar con WebDriverWait
# Ejemplo: element_to_be_clickable, presence_of_element_located, etc.
from selenium.webdriver.support import expected_conditions as EC

# Service: Maneja el servicio del driver del navegador
from selenium.webdriver.chrome.service import Service

# ChromeDriverManager: Descarga y configura automáticamente el driver de Chrome
from webdriver_manager.chrome import ChromeDriverManager

# --- IMPORTACIONES PARA CIENCIA DE DATOS ---

# pandas: Manipulación de datos en formato tabular (DataFrames)
# Lo usaremos para almacenar y procesar los datos scrapeados
import pandas as pd

# numpy: Operaciones numéricas y arrays
# Complementa pandas para análisis numérico
import numpy as np

# time: Control de tiempos y pausas
# Útil para dar tiempo a que las páginas carguen completamente
import time

print("Todas las librerías importadas correctamente")

# CONSEJO PEDAGÓGICO:
# Si alguna importación falla, significa que la librería no está instalada.
# Vuelve a las celdas anteriores y ejecuta los pip install correspondientes.

---

## 3. Conceptos Básicos <a name="conceptos-basicos"></a>

### 3.1 Inicializar el navegador

El primer paso es crear una instancia del navegador:

In [ ]:
# ========================================
# CELDA 4: INICIALIZAR EL NAVEGADOR CHROME
# ========================================

# PASO 1: Configurar las opciones de Chrome
# ChromeOptions() permite personalizar cómo se ejecuta el navegador
options = webdriver.ChromeOptions()

# OPCIÓN 1: Modo headless (sin interfaz gráfica)
# Descomenta la siguiente línea si quieres que el navegador se ejecute en segundo plano
# Útil para servidores o cuando no necesitas ver el navegador
# options.add_argument('--headless')

# OPCIÓN 2: Desactivar detección de automatización
# Algunos sitios detectan que Selenium está controlando el navegador
# Esta opción ayuda a que el navegador parezca "normal"
options.add_argument('--disable-blink-features=AutomationControlled')

# OPCIÓN 3: Iniciar maximizado
# Abre el navegador en pantalla completa
# Útil para asegurar que todos los elementos sean visibles
options.add_argument('--start-maximized')

# PASO 2: Inicializar el driver (el controlador del navegador)
# Esto ABRE el navegador Chrome y te da control sobre él
driver = webdriver.Chrome(
    # Service: Configura el servicio del driver
    # ChromeDriverManager().install(): Descarga automáticamente el driver correcto
    service=Service(ChromeDriverManager().install()),
    
    # Aplica las opciones que configuramos arriba
    options=options
)

print("✅ Navegador Chrome iniciado correctamente")

# QUÉ ESTÁ PASANDO EN SEGUNDO PLANO:
# 1. ChromeDriverManager verifica tu versión de Chrome
# 2. Descarga el chromedriver compatible (si no lo tiene)
# 3. Chrome se abre y queda bajo control de Python
# 4. Ahora puedes enviar comandos al navegador usando 'driver'

# IMPORTANTE PARA LA CLASE:
# - 'driver' es tu ROBOT que controla el navegador
# - Todo lo que hagas con Selenium usa este objeto 'driver'
# - Ejemplo: driver.get(), driver.find_element(), etc.

### 3.2 Abrir una página web

In [ ]:
# ========================================
# CELDA 5: NAVEGAR A UNA PÁGINA WEB
# ========================================

# PASO 1: Definir la URL que queremos visitar
# En este caso, vamos a Wikipedia (sitio perfecto para practicar web scraping)
url = "https://www.wikipedia.org"

# PASO 2: Usar driver.get() para navegar a la URL
# Esto es equivalente a escribir la URL en la barra de direcciones y presionar Enter
driver.get(url)

# QUÉ HACE driver.get():
# 1. Navega a la URL especificada
# 2. Espera a que la página cargue completamente (carga inicial del HTML)
# 3. Devuelve el control una vez cargada

# PASO 3: Obtener el título de la página
# La propiedad driver.title devuelve el contenido de la etiqueta <title> del HTML
# Esto es útil para verificar que estamos en la página correcta
print(f"Título de la página: {driver.title}")

# PASO 4: Obtener la URL actual
# driver.current_url devuelve la URL en la que estamos ahora
# Puede ser diferente a la URL original si hubo redirecciones
print(f"URL actual: {driver.current_url}")

# APLICACIÓN EN CIENCIA DE DATOS:
# Verificar el título y URL nos ayuda a:
# - Confirmar que la navegación fue exitosa
# - Detectar redirecciones inesperadas
# - Validar que estamos en la página correcta antes de extraer datos
# - Debugging: si algo falla, sabemos dónde estamos realmente

### 3.3 Cerrar el navegador

**Importante:** Siempre cierra el navegador al terminar para liberar recursos.

In [ ]:
# Cerrar la pestaña actual
# driver.close()

# Cerrar el navegador completamente
# driver.quit()

print("💡 No ejecutes esto aún, lo usaremos al final")

---

## 4. Navegación Web <a name="navegacion"></a>

### 4.1 Operaciones básicas de navegación

In [ ]:
# Reiniciar el driver si es necesario
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

# Navegar a diferentes páginas
driver.get("https://www.wikipedia.org")
time.sleep(2)

driver.get("https://www.python.org")
time.sleep(2)

# Retroceder
driver.back()
time.sleep(1)
print(f"Después de retroceder: {driver.title}")

# Avanzar
driver.forward()
time.sleep(1)
print(f"Después de avanzar: {driver.title}")

# Refrescar la página
driver.refresh()
print("Página refrescada")

### 4.2 Capturas de pantalla

In [ ]:
# Tomar una captura de pantalla
driver.get("https://www.wikipedia.org")
time.sleep(2)

driver.save_screenshot("captura_wikipedia.png")
print("✅ Captura de pantalla guardada como 'captura_wikipedia.png'")

---

## 5. Localización de Elementos <a name="localizacion"></a>

### 5.1 Métodos de localización

Selenium ofrece varias formas de encontrar elementos en una página:

| Método | Descripción | Ejemplo |
|--------|-------------|----------|
| `By.ID` | Por el atributo id | `driver.find_element(By.ID, "search")` |
| `By.NAME` | Por el atributo name | `driver.find_element(By.NAME, "q")` |
| `By.CLASS_NAME` | Por la clase CSS | `driver.find_element(By.CLASS_NAME, "btn")` |
| `By.TAG_NAME` | Por la etiqueta HTML | `driver.find_element(By.TAG_NAME, "h1")` |
| `By.CSS_SELECTOR` | Por selector CSS | `driver.find_element(By.CSS_SELECTOR, "div.container")` |
| `By.XPATH` | Por expresión XPath | `driver.find_element(By.XPATH, "//div[@id='content']")` |

### 5.2 Ejemplos prácticos de localización

In [ ]:
# ========================================
# CELDA 6: LOCALIZAR ELEMENTOS EN LA PÁGINA
# ========================================

# CONTEXTO: Estamos en la página principal de Wikipedia
# Ahora vamos a aprender a ENCONTRAR elementos específicos en la página

# Navegar a Wikipedia
driver.get("https://www.wikipedia.org")
time.sleep(2)  # Pausa de 2 segundos para asegurar que la página cargó

# --- EJEMPLO 1: Localizar por ID ---
# El ID es único en toda la página, es la forma MÁS RÁPIDA y CONFIABLE
# Equivale a: document.getElementById("searchInput") en JavaScript

search_box = driver.find_element(By.ID, "searchInput")
print(f"✅ Caja de búsqueda encontrada: {search_box.tag_name}")

# EXPLICACIÓN:
# - find_element() busca UN SOLO elemento (el primero que encuentre)
# - By.ID indica que buscaremos por el atributo 'id' del HTML
# - "searchInput" es el valor del id que buscamos: <input id="searchInput">
# - tag_name nos dice qué tipo de elemento HTML es (probablemente 'input')


# --- EJEMPLO 2: Localizar por CSS Selector ---
# Los selectores CSS son muy potentes y flexibles
# Si conoces CSS, este método te resultará familiar

try:
    # Intentar varios selectores que Wikipedia usa
    logo = driver.find_element(By.CSS_SELECTOR, ".central-featured")
    print(f"✅ Sección central encontrada")
except:
    print("⚠️ Selector específico no encontrado (la estructura puede variar)")

# EXPLICACIÓN:
# - ".central-featured" busca elementos con la clase CSS "central-featured"
# - El punto (.) indica que es una clase
# - CSS Selector puede ser muy específico: "div.clase #id > p"
# - IMPORTANTE: Los selectores pueden cambiar si el sitio actualiza su diseño


# --- EJEMPLO 3: Localizar MÚLTIPLES elementos ---
# find_elements() (plural) encuentra TODOS los elementos que coinciden

# Buscar todos los enlaces en la página principal
language_links = driver.find_elements(By.CSS_SELECTOR, "a.link-box")
if len(language_links) > 0:
    print(f"✅ Se encontraron {len(language_links)} enlaces de idiomas")
else:
    # Plan B: buscar todos los enlaces en general
    all_links = driver.find_elements(By.TAG_NAME, "a")
    print(f"✅ Se encontraron {len(all_links)} enlaces en total en la página")

# DIFERENCIA CLAVE:
# - find_element()  -> Devuelve 1 elemento (o error si no existe)
# - find_elements() -> Devuelve una LISTA de elementos (puede ser vacía)

# APLICACIÓN EN CIENCIA DE DATOS:
# - Usar find_elements() para extraer múltiples productos, artículos, precios, etc.
# - Luego iterar sobre la lista para procesar cada elemento
# - Ejemplo: extraer todos los títulos de noticias, todos los precios de productos

# CONSEJO PEDAGÓGICO:
# Para encontrar el selector correcto:
# 1. Abre las DevTools del navegador (F12)
# 2. Click en el inspector de elementos (icono de flecha)
# 3. Click en el elemento que quieres
# 4. Click derecho en el HTML -> Copy -> Copy selector

# IMPORTANTE: Siempre usa try-except cuando busques elementos específicos
# porque los sitios web cambian frecuentemente su estructura

### 5.3 Interactuar con elementos

In [ ]:
# ========================================
# CELDA 7: INTERACTUAR CON ELEMENTOS (ESCRIBIR)
# ========================================

# Una vez que ENCONTRAMOS un elemento, podemos INTERACTUAR con él
# Las interacciones más comunes son: escribir texto, hacer click, enviar formularios

# PASO 1: Localizar el campo de búsqueda
search_box = driver.find_element(By.ID, "searchInput")

# PASO 2: Limpiar el campo (por si tiene texto previo)
search_box.clear()
# clear() vacía completamente el campo de texto
# Buena práctica: siempre limpiar antes de escribir

# PASO 3: Escribir texto en el campo
search_box.send_keys("Python programming")
print("✅ Texto ingresado en la búsqueda")

# EXPLICACIÓN de send_keys():
# - Simula que un usuario escribe en el teclado
# - Escribe carácter por carácter (como un humano)
# - Puede recibir texto normal o teclas especiales (Keys.ENTER, Keys.TAB, etc.)
# - Es más realista que simplemente cambiar el valor del campo


# PASO 4: Enviar el formulario presionando Enter
search_box.send_keys(Keys.RETURN)
# Keys.RETURN simula presionar la tecla Enter
# Esto envía el formulario de búsqueda

time.sleep(3)  # Esperar 3 segundos para que cargue la página de resultados

print(f"Navegado a: {driver.title}")

# ALTERNATIVAS para enviar formularios:
# 1. send_keys(Keys.RETURN) - Presionar Enter (lo que usamos aquí)
# 2. elemento.submit() - Enviar el formulario directamente
# 3. Hacer click en el botón de búsqueda

# APLICACIÓN EN CIENCIA DE DATOS:
# - Automatizar búsquedas: buscar múltiples términos y recopilar resultados
# - Rellenar formularios: extraer datos de sitios que requieren login
# - Filtrar datos: seleccionar fechas, categorías, etc. antes de extraer
# 
# EJEMPLO DE USO REAL:
# for termino in ["Python", "Machine Learning", "Data Science"]:
#     search_box.clear()
#     search_box.send_keys(termino)
#     search_box.send_keys(Keys.RETURN)
#     # ... extraer resultados ...
#     # ... guardar en DataFrame ...

### 5.4 Hacer click en elementos

In [ ]:
# Regresar a la página principal
driver.get("https://www.wikipedia.org")
time.sleep(2)

# Encontrar y hacer click en el enlace de español
try:
    # Método 1: Intentar por ID específico
    spanish_link = driver.find_element(By.XPATH, "//a[@id='js-link-box-es']")
    spanish_link.click()
    time.sleep(2)
    print(f"✅ Click exitoso. Ahora en: {driver.title}")
except:
    try:
        # Método 2: Buscar el enlace que contiene "Español"
        spanish_link = driver.find_element(By.XPATH, "//a[contains(@href, 'es.wikipedia.org')]")
        spanish_link.click()
        time.sleep(2)
        print(f"✅ Click exitoso usando método alternativo. Ahora en: {driver.title}")
    except Exception as e:
        print(f"⚠️ No se pudo hacer click en el enlace: {e}")
        print("Esto puede pasar si Wikipedia cambió su estructura HTML")

# EXPLICACIÓN DE LOS MÉTODOS:
# Método 1: Busca por ID exacto (más específico pero puede cambiar)
# Método 2: Busca cualquier enlace que contenga 'es.wikipedia.org' (más flexible)
#
# LECCIÓN IMPORTANTE:
# Siempre ten un Plan B cuando trabajes con selectores
# Los sitios web actualizan su código HTML frecuentemente

---

## 6. Extracción de Datos <a name="extraccion"></a>

### 6.1 Extraer texto de elementos

In [ ]:
# Navegar a la Wikipedia en español
driver.get("https://es.wikipedia.org")
time.sleep(2)

# Extraer el título principal
try:
    # Intentar método 1: por clase específica
    titulo = driver.find_element(By.CSS_SELECTOR, "img.central-featured-logo")
    print(f"Título (alt text): {titulo.get_attribute('alt')}")
except:
    try:
        # Método 2: por el título de la página
        print(f"Título de la página: {driver.title}")
    except Exception as e:
        print(f"⚠️ Error al extraer título: {e}")

# Extraer artículos destacados
# NOTA: La estructura de Wikipedia cambia, usaremos selectores más genéricos
try:
    # Buscar en la sección de artículo destacado
    featured_section = driver.find_element(By.ID, "mp-tfa")
    # Buscar enlaces dentro de esa sección
    featured_articles = featured_section.find_elements(By.TAG_NAME, "a")
    
    print("\n📰 Enlaces en artículo destacado:")
    for article in featured_articles[:3]:  # Primeros 3 enlaces
        if article.text.strip():  # Solo mostrar si tiene texto
            print(f"  - {article.text}")
except:
    print("\n⚠️ No se encontró la sección de artículos destacados")
    print("Esto es normal: Wikipedia cambia su estructura frecuentemente")

# LECCIÓN IMPORTANTE PARA LA CLASE:
# En web scraping real, SIEMPRE debes:
# 1. Usar try-except para manejar cambios en la estructura
# 2. Tener métodos alternativos de extracción
# 3. Verificar periódicamente que tus scrapers sigan funcionando
# 4. Usar selectores lo más genéricos posible (cuando sea apropiado)

### 6.2 Extraer atributos de elementos

In [ ]:
# ========================================
# EXTRAER ENLACES Y ATRIBUTOS
# ========================================

# Buscar enlaces en la sección principal de Wikipedia
try:
    # Buscar en la sección de artículo destacado
    tfa_section = driver.find_element(By.ID, "mp-tfa")
    links = tfa_section.find_elements(By.TAG_NAME, "a")
except:
    # Si no encuentra esa sección, buscar en toda la página
    links = driver.find_elements(By.CSS_SELECTOR, "#content a")

print("🔗 Enlaces encontrados:")

# Contador para limitar la salida
count = 0
for link in links:
    texto = link.text.strip()
    url = link.get_attribute("href")
    
    # Solo mostrar enlaces con texto y URL válida
    if texto and url and count < 5:  # Limitar a 5 para no saturar la salida
        print(f"  Texto: {texto}")
        print(f"  URL: {url}\n")
        count += 1

if count == 0:
    print("  ⚠️ No se encontraron enlaces con texto en esta sección")

# CONCEPTOS CLAVE:
# - .text -> Obtiene el TEXTO VISIBLE del elemento
# - .get_attribute("href") -> Obtiene el ATRIBUTO href (la URL del enlace)
# - Otros atributos útiles: "class", "id", "src" (para imágenes), "value" (para inputs)

# APLICACIÓN EN CIENCIA DE DATOS:
# Extraer:
# - URLs de productos para visitar después
# - Precios de elementos (get_attribute("data-price"))
# - IDs únicos para seguimiento
# - Clases CSS para categorización

### 6.3 Esperas explícitas (muy importante)

Las esperas explícitas son cruciales para trabajar con contenido dinámico:

In [ ]:
# ========================================
# CELDA 8: ESPERAS EXPLÍCITAS (MUY IMPORTANTE)
# ========================================

# ⚠️ PROBLEMA CON time.sleep():
# - time.sleep(5) siempre espera 5 segundos, aunque la página cargue en 1 segundo
# - Si la página tarda más de 5 segundos, el código fallará
# - Es INEFICIENTE y NO CONFIABLE

# ✅ SOLUCIÓN: ESPERAS EXPLÍCITAS (WebDriverWait)
# - Espera SOLO hasta que se cumpla una condición
# - Si la condición se cumple antes, continúa inmediatamente
# - Si no se cumple en el tiempo límite, lanza una excepción
# - Es EFICIENTE y CONFIABLE

driver.get("https://es.wikipedia.org")

try:
    # EJEMPLO 1: Esperar hasta que un elemento EXISTA en el DOM
    # WebDriverWait(driver, 10) -> Espera MÁXIMO 10 segundos
    # until() -> Espera hasta que la condición sea True
    # presence_of_element_located -> El elemento existe en el HTML
    
    search_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "searchInput"))
    )
    print("✅ Elemento de búsqueda encontrado con espera explícita")
    
    # QUÉ HACE ESTO:
    # 1. Busca el elemento cada 0.5 segundos (por defecto)
    # 2. Si lo encuentra, devuelve el elemento inmediatamente
    # 3. Si pasan 10 segundos y no lo encuentra, lanza TimeoutException
    
    
    # EJEMPLO 2: Esperar hasta que un elemento sea CLICKEABLE
    # element_to_be_clickable verifica que:
    # - El elemento existe
    # - Es visible
    # - Está habilitado (no disabled)
    
    search_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))
    )
    print("✅ Botón de búsqueda es clickeable")
    
except Exception as e:
    print(f"Error en espera: {e}")

# OTRAS CONDICIONES ÚTILES:
# - visibility_of_element_located: Elemento visible (no oculto con CSS)
# - invisibility_of_element_located: Elemento NO visible
# - text_to_be_present_in_element: Texto específico presente
# - element_to_be_selected: Checkbox/radio seleccionado
# - staleness_of: Elemento ya no está en el DOM

# POR QUÉ ES CRUCIAL EN CIENCIA DE DATOS:
# - Sitios modernos cargan datos con JavaScript (AJAX)
# - Los datos pueden tardar tiempo en aparecer
# - Sin esperas explícitas, extraerás datos incompletos o vacíos
# - Las esperas explícitas garantizan que los datos estén listos

# REGLA DE ORO:
# ❌ NO uses time.sleep() para esperar elementos
# ✅ USA WebDriverWait con expected_conditions

---

## 7. Web Scraping Avanzado <a name="scraping-avanzado"></a>

### 7.1 Scroll en la página

Muchas páginas cargan contenido al hacer scroll:

In [ ]:
# Navegar a una página
# Abre la URL de Wikipedia sobre Python en el navegador controlado por Selenium
driver.get("https://es.wikipedia.org/wiki/Python")
# Pausa la ejecución durante 2 segundos para permitir que la página cargue completamente
time.sleep(2)

# Scroll hacia abajo
# Ejecuta código JavaScript que desplaza la ventana hasta el final de la página
# scrollHeight obtiene la altura total del documento
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
# Espera 1 segundo después del desplazamiento
time.sleep(1)
print("✅ Scroll hacia abajo completado")

# Scroll hacia arriba
# Ejecuta JavaScript para volver al inicio de la página (posición 0,0)
driver.execute_script("window.scrollTo(0, 0);")
# Pausa de 1 segundo para visualizar el movimiento
time.sleep(1)
print("✅ Scroll hacia arriba completado")

# Scroll a un elemento específico
try:
    # Intenta localizar un elemento HTML con el ID "Historia"
    elemento = driver.find_element(By.ID, "Historia")
    # Ejecuta JavaScript para hacer scroll hasta que el elemento sea visible
    # scrollIntoView(true) alinea el elemento en la parte superior de la ventana
    driver.execute_script("arguments[0].scrollIntoView(true);", elemento)
    # Espera 1 segundo después del desplazamiento
    time.sleep(1)
    print("✅ Scroll a elemento específico completado")
except:
    # Si el elemento no existe o hay algún error, muestra este mensaje
    print("Elemento no encontrado")

### 7.2 Manejar múltiples pestañas

In [ ]:
# Abrir una nueva pestaña
driver.execute_script("window.open('https://www.python.org', '_blank');")
time.sleep(2)

# Obtener todas las pestañas
tabs = driver.window_handles
print(f"Número de pestañas abiertas: {len(tabs)}")

# Cambiar a la segunda pestaña
driver.switch_to.window(tabs[1])
print(f"Pestaña activa: {driver.title}")
time.sleep(2)

# Volver a la primera pestaña
driver.switch_to.window(tabs[0])
print(f"Pestaña activa: {driver.title}")

# Cerrar la segunda pestaña
driver.switch_to.window(tabs[1])
driver.close()
driver.switch_to.window(tabs[0])
print("✅ Segunda pestaña cerrada")

### 7.3 Extraer tablas HTML a DataFrame

In [ ]:
# ========================================
# CELDA 9: EXTRAER TABLAS HTML A DATAFRAME
# ========================================

# 🎯 OBJETIVO: Convertir una tabla HTML en un DataFrame de pandas
# Esto es FUNDAMENTAL en ciencia de datos porque muchos datos están en tablas web

# PASO 1: Navegar a una página con tablas
driver.get("https://es.wikipedia.org/wiki/Anexo:Pa%C3%ADses_por_poblaci%C3%B3n")
time.sleep(3)  # Dar tiempo a que cargue la tabla

try:
    # PASO 2: Localizar la tabla usando CSS Selector
    # "table.wikitable" busca una tabla con la clase "wikitable"
    tabla = driver.find_element(By.CSS_SELECTOR, "table.wikitable")
    
    # PASO 3: Extraer los ENCABEZADOS de la tabla
    encabezados = []
    # Las etiquetas <th> contienen los encabezados de las columnas
    headers = tabla.find_elements(By.TAG_NAME, "th")
    
    for header in headers[:5]:  # Solo los primeros 5 encabezados
        # .strip() elimina espacios en blanco al inicio y final
        encabezados.append(header.text.strip())
    
    # QUÉ HICIMOS:
    # - Encontramos todos los <th> dentro de la tabla
    # - Extraemos el texto de cada uno
    # - Los guardamos en una lista que será el header del DataFrame
    
    
    # PASO 4: Extraer las FILAS de datos
    filas = []
    # Las etiquetas <tr> representan filas (table row)
    rows = tabla.find_elements(By.TAG_NAME, "tr")
    
    # Iteramos sobre las filas (saltamos la primera que son encabezados)
    for row in rows[1:11]:  # Filas 1 a 10 (primeros 10 países)
        # Las etiquetas <td> son las celdas de datos (table data)
        celdas = row.find_elements(By.TAG_NAME, "td")
        
        # Verificar que la fila tiene suficientes celdas
        if len(celdas) >= 3:
            # Extraer el texto de cada celda (primeras 5 columnas)
            fila_datos = [celda.text.strip() for celda in celdas[:5]]
            # Añadir la fila a nuestra lista
            filas.append(fila_datos)
    
    # ESTRUCTURA HTML DE UNA TABLA:
    # <table>
    #   <tr>                    <- Fila
    #     <th>Encabezado 1</th> <- Celda de encabezado
    #     <th>Encabezado 2</th>
    #   </tr>
    #   <tr>
    #     <td>Dato 1</td>        <- Celda de datos
    #     <td>Dato 2</td>
    #   </tr>
    # </table>
    
    
    # PASO 5: Crear el DataFrame de pandas
    df = pd.DataFrame(filas, columns=encabezados if encabezados else None)
    
    print("✅ Tabla extraída exitosamente\\n")
    print(df.head())
    
    # AHORA TIENES UN DATAFRAME CON LOS DATOS DE LA TABLA WEB!
    # Puedes hacer análisis, limpiar datos, exportar a CSV, etc.
    
except Exception as e:
    print(f"Error al extraer tabla: {e}")

# APLICACIONES EN CIENCIA DE DATOS:
# 1. Extraer tablas de estadísticas deportivas
# 2. Recopilar datos financieros de sitios de bolsa
# 3. Obtener rankings de universidades, empresas, etc.
# 4. Compilar datos demográficos de múltiples fuentes
# 5. Crear datasets para Machine Learning

# SIGUIENTE PASO TÍPICO:
# df.to_csv('paises_poblacion.csv', index=False)  # Guardar a CSV
# df.describe()  # Análisis estadístico
# df.plot()  # Visualización

---

## 8. Caso Práctico Completo <a name="caso-practico"></a>

### Proyecto: Extraer datos de búsqueda de Wikipedia

Vamos a crear un scraper que:
1. Busca un término en Wikipedia
2. Extrae información de los resultados
3. Guarda los datos en un DataFrame
4. Exporta a CSV

In [ ]:
def buscar_wikipedia(termino_busqueda, num_resultados=5):
    """
    ========================================
    CASO PRÁCTICO COMPLETO: SCRAPER DE WIKIPEDIA
    ========================================
    
    Esta función demuestra un FLUJO COMPLETO de web scraping para ciencia de datos:
    1. Configurar el navegador
    2. Navegar a un sitio web
    3. Realizar una búsqueda
    4. Extraer datos estructurados
    5. Limpiar y organizar los datos
    6. Retornar un DataFrame listo para análisis
    
    Args:
        termino_busqueda (str): Término que queremos investigar
        num_resultados (int): Cantidad de resultados a extraer (no usado en esta versión)
    
    Returns:
        pd.DataFrame: DataFrame con información estructurada del artículo
    """
    
    # ==========================================
    # PASO 1: CONFIGURACIÓN DEL NAVEGADOR
    # ==========================================
    
    options = webdriver.ChromeOptions()
    
    # Opción anti-detección: algunos sitios bloquean bots
    options.add_argument('--disable-blink-features=AutomationControlled')
    
    # Maximizar ventana para asegurar que todos los elementos sean visibles
    options.add_argument('--start-maximized')
    
    # Inicializar el driver con configuración automática
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )
    
    try:
        # ==========================================
        # PASO 2: NAVEGACIÓN Y BÚSQUEDA
        # ==========================================
        
        # Navegar a Wikipedia en español
        driver.get("https://es.wikipedia.org")
        print(f"🔍 Buscando: {termino_busqueda}")
        
        # ESPERA EXPLÍCITA: Asegurar que el campo de búsqueda existe
        # Esto es MÁS CONFIABLE que time.sleep()
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "searchInput"))
        )
        
        # Limpiar y escribir el término de búsqueda
        search_box.clear()
        search_box.send_keys(termino_busqueda)
        search_box.send_keys(Keys.RETURN)  # Enviar formulario
        
        # Esperar a que la página de resultados cargue
        time.sleep(3)
        
        # ==========================================
        # PASO 3: EXTRACCIÓN DE DATOS
        # ==========================================
        
        # Inicializar lista para almacenar los datos
        datos = []
        
        # --- EXTRAER TÍTULO DEL ARTÍCULO ---
        try:
            # El título principal está en un <h1> con id "firstHeading"
            titulo = driver.find_element(By.ID, "firstHeading").text
        except:
            # Si no se encuentra, usar valor por defecto
            titulo = "No disponible"
        
        
        # --- EXTRAER PRIMER PÁRRAFO (RESUMEN) ---
        try:
            # Encontrar TODOS los párrafos en el contenido principal
            parrafos = driver.find_elements(By.CSS_SELECTOR, "#mw-content-text p")
            primer_parrafo = ""
            
            # Buscar el primer párrafo con contenido significativo
            for p in parrafos:
                texto = p.text.strip()
                # Filtrar párrafos vacíos o muy cortos
                if len(texto) > 50:
                    # Limitar a 200 caracteres para mantener el DataFrame manejable
                    primer_parrafo = texto[:200] + "..."
                    break
        except:
            primer_parrafo = "No disponible"
        
        
        # --- EXTRAER NÚMERO DE SECCIONES ---
        try:
            # Los encabezados de sección tienen la clase "mw-headline"
            secciones = driver.find_elements(By.CSS_SELECTOR, ".mw-headline")
            num_secciones = len(secciones)
            # Este dato nos indica la "profundidad" del artículo
        except:
            num_secciones = 0
        
        
        # --- EXTRAER CATEGORÍAS ---
        try:
            # Las categorías están en la parte inferior del artículo
            categorias = driver.find_elements(By.CSS_SELECTOR, "#mw-normal-catlinks ul li")
            # Extraer solo las primeras 3 categorías
            lista_categorias = [cat.text for cat in categorias[:3]]
            # Unirlas en un string separado por comas
            categorias_texto = ", ".join(lista_categorias)
        except:
            categorias_texto = "No disponible"
        
        
        # ==========================================
        # PASO 4: ESTRUCTURAR LOS DATOS
        # ==========================================
        
        # Crear un diccionario con los datos extraídos
        # Esto facilita la conversión a DataFrame
        datos.append({
            'Título': titulo,
            'Descripción': primer_parrafo,
            'Número de secciones': num_secciones,
            'Categorías': categorias_texto,
            'URL': driver.current_url  # URL final (puede haber habido redirección)
        })
        
        
        # ==========================================
        # PASO 5: CONVERTIR A DATAFRAME
        # ==========================================
        
        # pandas.DataFrame convierte nuestra lista de diccionarios en una tabla
        df = pd.DataFrame(datos)
        
        print(f"✅ Extracción completada: {len(datos)} resultados")
        
        return df
        
    except Exception as e:
        # MANEJO DE ERRORES: Si algo falla, mostrar el error
        print(f"❌ Error: {e}")
        # Retornar DataFrame vacío para evitar que el programa se detenga
        return pd.DataFrame()
    
    finally:
        # ==========================================
        # PASO 6: LIMPIEZA (SIEMPRE SE EJECUTA)
        # ==========================================
        
        # IMPORTANTE: Siempre cerrar el navegador para liberar recursos
        # finally: se ejecuta SIEMPRE, incluso si hubo un error
        driver.quit()

# ==========================================
# EJECUTAR LA FUNCIÓN
# ==========================================

# Buscar información sobre "Inteligencia Artificial"
df_resultados = buscar_wikipedia("Inteligencia Artificial")

print("\\n📊 Resultados:")
print(df_resultados)

# EXPLICACIÓN PEDAGÓGICA DEL FLUJO:
# 1. Configuramos el navegador con opciones específicas
# 2. Navegamos a Wikipedia y realizamos una búsqueda
# 3. Esperamos a que cargue la página (con esperas explícitas)
# 4. Extraemos múltiples tipos de datos del artículo
# 5. Estructuramos los datos en un diccionario
# 6. Convertimos a DataFrame para análisis
# 7. Cerramos el navegador (limpieza de recursos)

# PRÓXIMOS PASOS:
# - Guardar el DataFrame en CSV
# - Analizar los datos con pandas
# - Crear visualizaciones
# - Escalar para buscar múltiples términos

### Guardar los resultados en CSV

In [ ]:
# Guardar en CSV
if not df_resultados.empty:
    df_resultados.to_csv('resultados_wikipedia.csv', index=False, encoding='utf-8-sig')
    print("✅ Datos guardados en 'resultados_wikipedia.csv'")
    
    # Mostrar información del DataFrame
    print("\n📈 Información del dataset:")
    print(df_resultados.info())
else:
    print("❌ No hay datos para guardar")

---

## 9. Mejores Prácticas <a name="mejores-practicas"></a>

### 9.1 Consejos importantes

1. **Siempre usa esperas explícitas** en lugar de `time.sleep()`
2. **Cierra el navegador** al terminar con `driver.quit()`
3. **Usa try-except** para manejar errores
4. **Respeta los robots.txt** de los sitios web
5. **No sobrecargues los servidores** - añade pausas razonables
6. **Usa headless mode** para mejor rendimiento en producción

### 9.2 Función completa con mejores prácticas

In [ ]:
# ========================================
# MEJORES PRÁCTICAS: CONTEXT MANAGER
# ========================================

# PROBLEMA COMÚN:
# Si olvidas llamar driver.quit(), el navegador queda abierto consumiendo memoria
# Si hay un error en el código, driver.quit() puede no ejecutarse

# SOLUCIÓN PROFESIONAL: Context Manager (with statement)

from contextlib import contextmanager

@contextmanager
def selenium_driver(headless=False):
    """
    Context Manager para gestionar automáticamente el ciclo de vida del driver.
    
    ¿QUÉ ES UN CONTEXT MANAGER?
    Es un patrón de diseño que garantiza:
    - Inicialización: Se ejecuta al entrar en el bloque 'with'
    - Limpieza: Se ejecuta SIEMPRE al salir, incluso si hay errores
    
    VENTAJAS:
    - No puedes olvidar cerrar el navegador
    - El navegador se cierra automáticamente, incluso con errores
    - Código más limpio y profesional
    - Previene fugas de memoria
    
    Args:
        headless (bool): Si True, ejecuta Chrome sin interfaz gráfica (más rápido)
    
    Yields:
        webdriver: Instancia del driver de Chrome lista para usar
    """
    
    # CONFIGURACIÓN DE OPCIONES
    options = webdriver.ChromeOptions()
    
    # Si headless=True, el navegador se ejecuta en segundo plano (sin ventana)
    if headless:
        options.add_argument('--headless')
        # VENTAJA: Más rápido, ideal para producción o servidores
        # DESVENTAJA: No puedes ver qué está haciendo
    
    # Opciones adicionales para mayor estabilidad
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--start-maximized')
    options.add_argument('--disable-gpu')  # Desactivar GPU (útil en servidores)
    options.add_argument('--no-sandbox')   # Mayor compatibilidad en Linux
    
    # INICIALIZAR EL DRIVER
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )
    
    try:
        # YIELD: Devuelve el driver al bloque 'with'
        # El código dentro del 'with' se ejecuta aquí
        yield driver
        
    finally:
        # LIMPIEZA: Se ejecuta SIEMPRE al salir del bloque 'with'
        # Incluso si hubo una excepción en el código
        driver.quit()
        print("✅ Navegador cerrado correctamente")


# ==========================================
# EJEMPLO DE USO
# ==========================================

# USO TRADICIONAL (menos seguro):
# driver = webdriver.Chrome(...)
# driver.get("https://...")
# # ... hacer cosas ...
# driver.quit()  # ¿Y si hay un error antes de esto?

# USO CON CONTEXT MANAGER (recomendado):
with selenium_driver(headless=False) as driver:
    driver.get("https://www.wikipedia.org")
    print(f"Título: {driver.title}")
    # Al salir del bloque 'with', driver.quit() se llama automáticamente

# FLUJO DE EJECUCIÓN:
# 1. Se ejecuta el código antes de 'yield' (inicialización)
# 2. Se ejecuta el código dentro del bloque 'with'
# 3. Se ejecuta el código en 'finally' (limpieza)
# 4. El navegador SIEMPRE se cierra, no importa qué pase

# ANALOGÍA PEDAGÓGICA:
# Es como abrir un archivo:
# with open('archivo.txt', 'r') as f:
#     contenido = f.read()
# # El archivo se cierra automáticamente

# APLICACIÓN EN CIENCIA DE DATOS:
# - Ejecutar múltiples scrapers sin preocuparte por la limpieza
# - Automatizar procesos en servidores (headless=True)
# - Código más robusto y mantenible
# - Prevenir errores de recursos no liberados

### 9.3 Manejo robusto de errores

In [ ]:
# ========================================
# MANEJO ROBUSTO DE ERRORES
# ========================================

# En web scraping, MUCHAS cosas pueden salir mal:
# - El elemento no existe
# - La página tarda en cargar
# - El sitio cambió su estructura
# - Problemas de conexión

# Es CRUCIAL manejar estos errores correctamente

from selenium.common.exceptions import (
    NoSuchElementException,      # Elemento no encontrado
    TimeoutException,             # Tiempo de espera agotado
    ElementNotInteractableException  # Elemento no interactuable
)

def extraer_elemento_seguro(driver, by, value, timeout=10):
    """
    Función ROBUSTA para extraer elementos con manejo profesional de errores.
    
    FILOSOFÍA:
    - "Espera lo mejor, prepárate para lo peor"
    - Nunca asumas que un elemento existirá
    - Siempre ten un plan B (valor por defecto)
    
    Args:
        driver: WebDriver de Selenium
        by: Método de localización (By.ID, By.CSS_SELECTOR, etc.)
        value: Valor del localizador (el selector específico)
        timeout: Tiempo máximo de espera en segundos (default: 10)
    
    Returns:
        str: Texto del elemento o mensaje de error descriptivo
    """
    try:
        # INTENTO 1: Esperar hasta que el elemento esté presente
        # WebDriverWait es más inteligente que time.sleep()
        elemento = WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((by, value))
        )
        return elemento.text
        
    except TimeoutException:
        # ERROR 1: El elemento no apareció en el tiempo límite
        # Posibles causas:
        # - Selector incorrecto
        # - Página muy lenta
        # - Elemento cargado con JavaScript que tardó más de lo esperado
        return f"⏱️ Timeout: Elemento no encontrado en {timeout} segundos"
        
    except NoSuchElementException:
        # ERROR 2: El elemento definitivamente no existe
        # Posibles causas:
        # - Selector erróneo
        # - Estructura del sitio cambió
        # - Estamos en la página equivocada
        return "❌ Elemento no existe en la página"
        
    except Exception as e:
        # ERROR 3: Cualquier otro error inesperado
        # Siempre es buena práctica tener un catch-all
        return f"❌ Error inesperado: {str(e)}"


# ==========================================
# DEMOSTRACIÓN DE USO
# ==========================================

with selenium_driver() as driver:
    driver.get("https://es.wikipedia.org")
    
    # --- CASO 1: Elemento que SÍ existe ---
    resultado1 = extraer_elemento_seguro(driver, By.ID, "searchInput")
    print(f"Resultado 1 (existe): {resultado1[:50] if len(resultado1) > 50 else resultado1}")
    
    # --- CASO 2: Elemento que NO existe ---
    resultado2 = extraer_elemento_seguro(driver, By.ID, "elemento_inexistente", timeout=3)
    print(f"Resultado 2 (no existe): {resultado2}")

# ==========================================
# PATRÓN TRY-EXCEPT EN WEB SCRAPING
# ==========================================

# PATRÓN RECOMENDADO para extracción de datos:

# datos = []
# for item in items:
#     try:
#         titulo = item.find_element(By.CSS_SELECTOR, "h2").text
#     except:
#         titulo = "No disponible"
#     
#     try:
#         precio = item.find_element(By.CSS_SELECTOR, ".precio").text
#     except:
#         precio = "No disponible"
#     
#     datos.append({
#         'titulo': titulo,
#         'precio': precio
#     })

# VENTAJAS DE ESTE ENFOQUE:
# 1. El scraper NO se detiene si falta un elemento
# 2. Recopilas TODOS los datos disponibles, aunque algunos falten
# 3. Puedes identificar patrones en los datos faltantes
# 4. En ciencia de datos, datos parciales > sin datos

# PRINCIPIOS CLAVE:
# ✅ Siempre usa try-except para extracciones
# ✅ Proporciona valores por defecto sensatos
# ✅ Registra los errores para debugging
# ✅ No dejes que un error detenga toda la recopilación
# ✅ Valida los datos después de extraerlos

# COMPARACIÓN:
# Código SIN manejo de errores:
#   precio = elemento.find_element(By.CLASS_NAME, "precio").text
#   -> Se detiene si el elemento no existe
#
# Código CON manejo de errores:
#   try:
#       precio = elemento.find_element(By.CLASS_NAME, "precio").text
#   except:
#       precio = "No disponible"
#   -> Continúa incluso si el elemento no existe

---

##  Ejercicios Propuestos

1. **Ejercicio 1:** Crea un scraper que extraiga los títulos de las noticias principales de un sitio de noticias

2. **Ejercicio 2:** Extrae una tabla de datos de Wikipedia y realiza un análisis básico con pandas

3. **Ejercicio 3:** Automatiza una búsqueda en Google y extrae los primeros 10 resultados

4. **Ejercicio 4:** Crea un scraper que navegue por múltiples páginas y compile datos en un CSV

---

##  Recursos Adicionales

- [Documentación oficial de Selenium](https://www.selenium.dev/documentation/)
- [Selenium con Python](https://selenium-python.readthedocs.io/)
- [XPath Cheat Sheet](https://devhints.io/xpath)
- [CSS Selectors Reference](https://www.w3schools.com/cssref/css_selectors.asp)

---

##  Conclusión

¡Felicidades! Has completado el tutorial de Selenium para Ciencia de Datos. Ahora conoces:

- ✅ Configuración y uso básico de Selenium
- ✅ Localización e interacción con elementos web
- ✅ Extracción de datos para análisis
- ✅ Técnicas avanzadas de web scraping
- ✅ Mejores prácticas y manejo de errores

### Próximos pasos:

1. Practica con diferentes sitios web
2. Combina Selenium con BeautifulSoup para parsing más eficiente
3. Aprende sobre proxies y rotación de user agents
4. Explora Scrapy para proyectos más grandes

**¡Happy Scraping! **